In [ ]:
import cv2
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import os
import numpy as np
import cv2
from google.colab.patches import cv2_imshow

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"

In [ ]:
drive_path = '/content/drive/MyDrive/'
folder_path = f'{drive_path}obj_detect/'
saved_model_path = f'{folder_path}saved_models/'

In [ ]:
def kmeans_np(np_img, k):

    npi = np_img.reshape((-1,3))

    pixel_vals = np.float32(npi)

    criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 100, 0.85)

    retval, labels, centers = cv2.kmeans(pixel_vals, k, None, criteria, 10, cv2.KMEANS_RANDOM_CENTERS)

    # convert data into 8-bit values
    centers = np.uint8(centers)
    segmented_data = centers[labels.flatten()]

    # reshape data into the original image dimensions
    segmented_image = segmented_data.reshape((np_img.shape))

    return segmented_image

In [ ]:
def image_grid(image, x_interval, y_interval):
  rows, cols, _ = image.shape
  for x in range(0, cols, x_interval):
      cv2.line(image, (x, 0), (x, rows), (255, 0, 0), 1)  # Blue lines for vertical grid

  for y in range(0, rows, y_interval):
      cv2.line(image, (0, y), (cols, y), (255, 0, 0), 1)  # Blue lines for horizontal grid


In [ ]:
def display_pic_2_col(double_pic_arrays, row_num):
  rows = []

  for i in range(row_num):
    row = np.hstack((double_pic_arrays[i][0], double_pic_arrays[i][1]))
    rows.append(row)
    grid = np.vstack(rows)

  cv2_imshow(grid)

In [ ]:
def print_boxes(img, bbox_list):
    if (type(img) == str):
        image = cv2.imread(img)
        # Convert the image from BGR to RGB (OpenCV uses BGR by default)
        image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    else:
        image_rgb = img


    # Create a figure and axis
    fig, ax = plt.subplots(1)

    # Display the image
    ax.imshow(image_rgb)

    for bbox in bbox_list:
        x_min, y_min, x_max, y_max = bbox
        width = x_max - x_min
        height = y_max - y_min

        # Create a rectangle patch
        rect = patches.Rectangle((x_min, y_min), width, height, linewidth=2, edgecolor='r', facecolor='none')

        # Add the patch to the axis
        ax.add_patch(rect)

    plt.show()

In [ ]:
def image_seg_from_path(pic_path, kmeans_cluster):

  # RV = Return Value
  RV = [None, None]

  pic = cv2.imread(pic_path)
  pic = cv2.resize(pic, (img_size, img_size))

  if (kmeans_cluster[0]): pic = kmeans_np(pic, kmeans_cluster[1])

  RV[0] = pic

  pic = pic.transpose(2, 0, 1)

  picTensor = torch.from_numpy(pic).float()
  picTensor = picTensor.to(device)

  segmentation = obj_detect(picTensor.unsqueeze(0))

  segmentation = segmentation.to('cpu')
  seg_np = segmentation.detach().numpy()
  seg_np = seg_np.squeeze().transpose(1, 2, 0)
  RV[1] = seg_np

  return RV

In [1]:
def accept_img(file_name, img_types = ['.png', '.jpg', '.jpeg']):
  for i in img_types:
    if (file_name.find(i) != -1):
      return True
  return False